In [93]:
import PyPDF2
import os
from constants import *
import string
import json
from difflib import SequenceMatcher


### Getting the data

In [94]:
# get printable characters
printable = set(string.printable)

metadataDict = {}
# iterate through directory where articles are stored
with open(metadataPath, 'r') as file:
    for i in range(10):
        line = next(file).strip()
        if json.loads(line)['id'] not in metadataDict:
            abstract = json.loads(line)['abstract']
            metadataDict[json.loads(line)['id']] = ''.join(filter(lambda x: x in printable, abstract))
    


In [98]:
articleDict = {}
# loop through each file in folder that contains articles
for fileName in os.listdir(articlesPath):
    print(fileName)
    with open(f'{articlesPath}/{fileName}', 'r', encoding='cp1252', errors='ignore') as file:
        path = os.path.basename(fileName)
        dictkey = path[0:9]
        articleDict[dictkey] = " ".join(file.readlines())


0704.0003v2.txt
0704.0007v1.txt
0704.0008v1.txt
0704.0009v1.txt
0704.0010v1.txt
0704.0012v1.txt


### Matching

In [107]:

def similar(a, b):
    # this method calculates the similarity score between two strings
    # Im removing spaces here to avoid spacing issues messing up similarity scores
    return SequenceMatcher(None, a.replace(" ", ""), b.replace(" ", "")).ratio()

def printMatch(metadataAbstract, maxMatchinString, similarity, articleID):
    print(f'Article ID: {articleID} ')
    print(f'matched abstract: \n {maxMatchingString} \n---- \noriginal abstract from metadata: \n {metadataAbstract} \n---- \nsimilarity(of the strings with no spaces): {similarity} \n')
    print(f'Length of matched abstract: {len(maxMatchingString)}')
    print(f'Length of original abstract: {len(metadataAbstract)}')
    print('------------------------------------------------------')

for articleID in articleDict:
    articleText = articleDict[articleID].replace('\n', "")
    if articleID in metadataDict:
        metadataAbstract = metadataDict[articleID].replace('\n', "")
    else:
        continue    
    i = 0
    maxMatchingString = ''
    maxMatch = 0
    abstractFound = False
    while i  < len(articleText):
        # find the abstract in the article, and move index to that location
        if articleText[i: i + 8].lower() == 'abstract':
            abstractFound = True
            i += 8
        elif not abstractFound:
            i += 1
            continue
        lengthMarginCounter = 0
        broken = False
        lengthMargin = 50
        # calculate similarities of strings with a margin of lengthMargin, meaning its length can be at most lengthMargin characters greater than the original
        # this heavily increases computation time but is able to match up to lengthMargin more characters 
        # 6 articles: value of 1 -> 1 second, value of 50 -> 25 seconds
        while lengthMarginCounter < lengthMargin:
            candidateAbstract = articleText[i:i + lengthMarginCounter + len(metadataAbstract)]
            similarity = similar(metadataAbstract, candidateAbstract)
            # if the similarity of the current candidate string is higher
            if similarity > maxMatch:
                maxMatchingString = candidateAbstract
                maxMatch = similarity
            # otherwise, if the similarity < .75 and the similarity score is not increasing, break 
            # (assuming that similarity score will increase as the abstract should be at beginning)
            elif similarity < .75:
                broken = True
                break
            lengthMarginCounter += 1
        if broken:
            break
        i += 1
    printMatch(metadataAbstract, maxMatchingString, maxMatch, articleID)


Article ID: 0704.0003 
matched abstract: 
   The evolution of Earth-Moon system is described by the dark matter field fluid model with a non-Newtonian approach proposed in the Meeting of Division of Particle and Field 2004, American Physical Society. The current behavior of the Earth-Moon system agrees with this model very well and the general pattern of the evolution of the Moon-Earth system described by this model agrees with geological and fossil evidence. The closest distance of the Moon to Earth was about 259000 km at 4.5 billion years ago, which is far beyond the Roche’s limit. The result suggests that the tidal friction may not be the primary cause for the evolution of the Earth-Moon system. The average dark matter field fluid constant derived from Earth-Moon system data is 4.39 × 10-22 s-1m-1. This model predicts that the Mars’s rotation is also slowing with the angular acceleration rate about -4.38 × 10-22 
---- 
original abstract from metadata: 
   The evolution of Earth-Moon